In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train, x_test = x_train[..., tf.newaxis]/255.0, x_test[..., tf.newaxis]/255.0

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Define a function to create the ConvNet model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation="relu" , input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')   
    ])
    return model

# Create a function for training and evaluating the model
def train_model(model, x_train, y_train, x_val, y_val, epochs=6, batch_size=32):
    model.compile(
        optimizer='adam',
        metrics=['accuracy'],
        loss = 'sparse_categorical_crossentropy'
    )
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=0)
    return history

# Create and train the model with different hyperparameters
models_histories = []
epochs = 10
batch_sizes = [32, 64, 128]
for batch_size in batch_sizes:
    model = create_model()
    history = train_model(model, x_train, y_train, x_val, y_val, epochs=epochs, batch_size=batch_size)
    models_histories.append(history)

# Plot training and validation accuracy for each model
plt.figure(figsize=(12, 8))
for i, history in enumerate(models_histories):
    plt.plot(history.history['accuracy'], label=f'Training Accuracy (Batch Size: {batch_sizes[i]})')
    plt.plot(history.history['val_accuracy'], label=f'Validation Accuracy (Batch Size: {batch_sizes[i]})')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model on the test set
model = create_model()
model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss = 'sparse_categorical_crossentropy'
)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_sizes[0], validation_data=(x_val, y_val), verbose=0)
loss, acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {acc}')
